In [131]:
import re
import statistics
import spacy
import string
import json
import nltk
import operator
import imdb
from collections import Counter
import collections

from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [137]:
# Red carpet – 
# For example, determine who was best dressed, worst dressed, most discussed, most controversial, 
# or perhaps find pictures of the best and worst dressed, etc.

In [133]:
nlp = spacy.load('en')
stop_words = set(stopwords.words('english')) 

gg_stop_words = ['HuffingtonPost','amp', 'Globe', 'RT', 'http', 'Golden', 'Globes', 'GoldenGlobes', 'Goldenglobes', 'Goldenglobe', 'gg','golden globes', 'golden globe', 'goldenglobe','goldenglobes','gg2015','gg15','goldenglobe2015','goldenglobe15','goldenglobes2015','goldenglobes15', 'gg2013','gg13','goldenglobe2013','goldenglobe13','goldenglobes2013','goldenglobes13', 'rt', '2013', '2015', '...', '`', 'MTVNews']

rc_stop_words = ['eredcarpet', '\'', 'lt', 'damn', '^']

gg_stop_words.extend(stop_words)
gg_stop_words.extend(rc_stop_words)

tweets = json.load(open("gg2013.json"))

clean_tweets = []

for tweet in tweets:
    text = tweet['text']
    text_tokens = word_tokenize(text)
    t = []
    sent = ''
    for w in text_tokens:
        if w.lower() not in gg_stop_words:
            if w not in string.punctuation: 
                no_url = re.sub(r'(//t.co.*|`)', '', w) # remove urls
                t.append(no_url)
    for i in range(len(t)): 
        sent += t[i] + ' '
    clean_tweets.append(sent)

In [134]:
# get all red carpet tweets
rc_tweets = []

for t in clean_tweets:
    if re.search(r'.*red.*carpet.*', t.lower()):
        rc_tweets.append(t)

In [79]:
# tweets_w_sentiment = {}

# for t in rc_tweets:
#     score = SentimentIntensityAnalyzer().polarity_scores(t)['compound']
#     if score != 0.0:
#         tweets_w_sentiment[t] = score

In [70]:
def get_names(text):
    article = nlp(text)
    labels = [x.label_ for x in article.ents]
    [(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]
    parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
    names = []
    for (key, value) in parts_of_speech.items() :
        if(value == "PERSON") :
            names.append(key)
    return names

In [105]:
def best_dressed():
    positive_sent = []
    for t in rc_tweets:
        score = SentimentIntensityAnalyzer().polarity_scores(t)['compound']
        if score > 0.5:
            positive_sent.append(t)
    count = {}
    for t in positive_sent: 
        names = get_names(t)
        for name in names :
            if len(name.split()) == 2:
                if name in count :
                    count[name] = count[name] + 1
                else:
                    count[name] = 1
#     winner = max(count.items(), key=operator.itemgetter(1))[0]
    winners = Counter(count).most_common(5)
    result = []
    for n in winners:
        result.append(n[0])
    return result

In [108]:
best_dressed()

['Zuhair Murad',
 'Anne Hathaway',
 'Jessica Chastain',
 'Jennifer Lawrence',
 'Taylor Swift']

In [128]:
def worst_dressed():
    
    neg_words = ['bad', 'ugly', 'worst', 'awful', 'horrible', 'hate', 'suck', 'sucks', 'sucked', 'disappointing']
    neg_sent = []
    for t in rc_tweets:
        if bool([ele for ele in neg_words if(ele in t)]):
            neg_sent.append(t)
    count = {}
    
    for t in neg_sent: 
        names = get_names(t)
        for name in names :
            if len(name.split()) == 2:
                if name in count :
                    count[name] = count[name] + 1
                else:
                    count[name] = 1
#     winner = max(count.items(), key=operator.itemgetter(1))[0]
    winners = Counter(count).most_common(5)
    result = []
    for n in winners:
        result.append(n[0])
    return result

In [129]:
worst_dressed()

['Taylor Swift', 'Kate Hudson', 'Amy Adams', 'Sally Fields', 'Lucy Liu']

In [135]:
def most_controversial():
    best = best_dressed()
    worst = worst_dressed()
    controversial = []
    for b in best:
        for w in worst:
            if b == w:
                controversial.append(w)
    return controversial
    

In [136]:
most_controversial()

['Taylor Swift']

In [140]:
def most_discussed():
    count = {}
    names = []
    for t in rc_tweets:
        names = get_names(t)
    for name in names :
        if len(name.split()) == 2:
            if name in count :
                count[name] = count[name] + 1
            else:
                count[name] = 1
    winner = max(count.items(), key=operator.itemgetter(1))[0]
    return winner

In [141]:
most_discussed()

'Jennifer Lawrence'